In [287]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [288]:
##Load the dataset
data = pd.read_csv("./Churn_Modelling.csv")
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [289]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [290]:
data.isnull().any()

RowNumber          False
CustomerId         False
Surname            False
CreditScore        False
Geography          False
Gender             False
Age                False
Tenure             False
Balance            False
NumOfProducts      False
HasCrCard          False
IsActiveMember     False
EstimatedSalary    False
Exited             False
dtype: bool

In [291]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [292]:
##Dropping features
data.drop(columns=["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [293]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [294]:
data["Gender"].unique()

array(['Female', 'Male'], dtype=object)

In [295]:
##Encoding categorical variables
label_encoder_gender = LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [296]:
"""
Here we will not use label encoder for Geography cause here we will
get 3 categories(0,1,2) and ANN is all about numerical calculations
and here it is considered that 2(Germany) is greater than 1(spain).
So we will use OnehotEncoding
"""

'\nHere we will not use label encoder for Geography cause here we will\nget 3 categories(0,1,2) and ANN is all about numerical calculations\nand here it is considered that 2(Germany) is greater than 1(spain).\nSo we will use OnehotEncoding\n'

In [297]:
print(type(data[["Geography"]]))

<class 'pandas.core.frame.DataFrame'>


In [298]:
from sklearn.preprocessing import OneHotEncoder
ohe_for_Geography = OneHotEncoder(sparse_output=False)
geo_encoder= ohe_for_Geography.fit_transform(data[["Geography"]])



In [299]:
ohe_for_Geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [300]:
##This matrix that i got after encoding, we need to make 
# this as dataframe
encoded_geo_df = pd.DataFrame(
    data=geo_encoder,
    columns=ohe_for_Geography.get_feature_names_out(['Geography'])
)
encoded_geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [301]:
##Combining all the columns
data_df = pd.concat(
    objs=[data.drop(["Geography"], axis=1), encoded_geo_df],
    axis=1
)
data_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [302]:
##Save the encoders
with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file=file)

with open("one_hot_encoder_geo.pkl", "wb") as file:
    pickle.dump(ohe_for_Geography, file=file)

In [303]:
## Divide the data into training and testing
x = data_df.drop(["Exited"], axis=1)
y = data_df["Exited"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [304]:
##Scaling these features
scaler = StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [305]:
##Saving this scaler too
with open("scaler_transform.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [306]:
##ANN implementation
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [307]:
##Building our ANN model
model = Sequential([
    Input(shape= (x_train.shape[1],)),
    Dense(64, activation='relu'),
    ##This is first connected hidden layer HL1
    Dense(32, activation= 'relu'),
    Dense(1, activation= 'sigmoid')##output layer
])

In [308]:
"""
You can also do this:
model = Sequential()
model.add(Input(shape=(10,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

"""

"\nYou can also do this:\nmodel = Sequential()\nmodel.add(Input(shape=(10,)))\nmodel.add(Dense(64, activation='relu'))\nmodel.add(Dense(1, activation='sigmoid'))\n\n"

In [309]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [310]:
##compile the model
model.compile(
    optimizer= 'adam',
    loss='binary_crossentropy',
    metrics = ['accuracy']
)

In [311]:
##Instead of this we can also do this:
'''opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

then you do:
model.compile(
    optimizer= opt,
    loss = loss, 
    metrics = ['accuracy']
)'''

"opt = tf.keras.optimizers.Adam(learning_rate=0.01)\nloss = tf.keras.losses.BinaryCrossentropy()\n\nthen you do:\nmodel.compile(\n    optimizer= opt,\n    loss = loss, \n    metrics = ['accuracy']\n)"

In [312]:
##Set up the Tensorboard
log_dir = "logs/fit/"+ datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)##Tensorboard is to visuallize
#all your logs while you train

In [313]:
##Setting ip early stopping
"""
Logic behind this is if we encounter such scenerio that in just 20
epochs, my model has trained at best level and after that its just
varying by 1or2 percent, Then its not necessary to train that model
for 100 epochs, main point behind this is if loss value is decreasing
after every epoch then continue but if its not decreasing after a
fixed number of epoch then stop. Early stopping allows us to do that
if the loss value is not decreasing significantly then just stop.
"""
early_stopping_callback = EarlyStopping(
    monitor= 'val_loss', # monitor validation loss 
    patience = 10, # be patience for atleast 5 epoch
    restore_best_weights =True

)

In [314]:
##Training model
history = model.fit(
    x_train, y_train, validation_data= (x_test, y_test), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7951 - loss: 0.4645 - val_accuracy: 0.8290 - val_loss: 0.3985
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8360 - loss: 0.3941 - val_accuracy: 0.8485 - val_loss: 0.3647
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8525 - loss: 0.3618 - val_accuracy: 0.8580 - val_loss: 0.3470
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8572 - loss: 0.3487 - val_accuracy: 0.8610 - val_loss: 0.3439
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8589 - loss: 0.3441 - val_accuracy: 0.8560 - val_loss: 0.3466
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8631 - loss: 0.3382 - val_accuracy: 0.8625 - val_loss: 0.3432
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8618 - loss: 0.3354 - val_accuracy: 0.8585 - val_loss: 0.3454
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8631 - loss: 0.3323 - val_accuracy: 0.8530

In [315]:
"""
You’re looking at the training log, and callbacks are influencing
 it, but they’re not the reason those lines appear. Let’s untangle
 this cleanly.

This part:

 accuracy: 0.8606 - loss: 0.3354 - val_accuracy: 0.8620 - val_loss: 0.3403

is printed by Keras itself during model.fit(), not specifically  
 by callbacks.
Keras shows this after every epoch because by default:
 model.fit(..., verbose=1)
So this is simply training progress output.

"""
##Then role of callnbacks
"""Then what are callbacks doing here?

You used:
 callbacks=[tensorflow_callback, early_stopping_callback]
They don’t print those metrics — they react to them.


a. EarlyStopping callback:
 It watches something like:
     monitor='val_loss'

So every time this line appears:
 val_loss: 0.3403

EarlyStopping checks:
✔ Is validation loss improving?
❌ Has it stopped improving for patience epochs?
If no improvement → training stops early

So those printed values are the signals EarlyStopping uses.




b. TensorBoard callback

Your tensorflow_callback is probably TensorBoard.

It takes those same numbers:

accuracy, loss, val_accuracy, val_loss


and logs them so you can see graphs like:

📈 Accuracy vs Epoch
📉 Loss vs Epoch
"""

"Then what are callbacks doing here?\n\nYou used:\n callbacks=[tensorflow_callback, early_stopping_callback]\nThey don’t print those metrics — they react to them.\n\n\na. EarlyStopping callback:\n It watches something like:\n     monitor='val_loss'\n\nSo every time this line appears:\n val_loss: 0.3403\n\nEarlyStopping checks:\n✔ Is validation loss improving?\n❌ Has it stopped improving for patience epochs?\nIf no improvement → training stops early\n\nSo those printed values are the signals EarlyStopping uses.\n\n\n\n\nb. TensorBoard callback\n\nYour tensorflow_callback is probably TensorBoard.\n\nIt takes those same numbers:\n\naccuracy, loss, val_accuracy, val_loss\n\n\nand logs them so you can see graphs like:\n\n📈 Accuracy vs Epoch\n📉 Loss vs Epoch\n"

In [316]:
model.save("model.keras")

In [317]:
##Launching tensorboard session
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [318]:
%tensorboard --logdir logs/fit/2026-02-01_14-29-55

Reusing TensorBoard on port 6010 (pid 13824), started 1:25:29 ago. (Use '!kill 13824' to kill it.)

In [ ]:
##Why 